In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from CTL.causal_tree_learn import CausalTree
import numpy as np
import pickle

In [2]:
print("### Load Data ######################")
data = pd.read_csv('data/preprocessed_app_data.csv')
data.head()

### Load Data ######################


,spark.executor.cores,spark.executor.memory,spark.default.parallelism,spark.serializer,spark.task.cpus,executorDeserializeCpuTime,executorRunTime,executorCpuTime,resultSize,jvmGcTime,Execution_Time
0,7,6,29,0,2,5.275688e+07,881.7425,5.304603e+08,1133.1900,251.3500,40424
1,7,6,37,0,1,5.040701e+07,1364.3075,5.896709e+08,1141.0900,234.2000,39742
2,6,6,16,1,2,5.691199e+07,852.4075,5.437210e+08,1302.3825,205.3500,41370
3,6,7,31,0,2,6.257915e+07,849.0325,5.488717e+08,1137.8575,219.5600,42098
4,5,1,10,1,2,5.772729e+07,7255.6350,3.820878e+09,1318.9675,1025.0025,384288


In [3]:
# columns = ['spark.executor.cores', 
#            'spark.executor.memory',
#            'spark.default.parallelism', 
#            'spark.serializer', 
#            'spark.task.cpus',
#            'executorDeserializeCpuTime', 
#            'executorRunTime', 
#            'executorCpuTime',
#            'resultSize', 
#            'jvmGcTime', 
#            'Execution_Time']


Configs Only

In [4]:
y = data['Execution_Time'].values
treatment = data['spark.serializer'].values
# # other configs
drop_cols = ['Execution_Time', #Outcome
             'spark.serializer', #Treatment
             'executorDeserializeCpuTime', 'executorRunTime', 'executorCpuTime', 'resultSize', 'jvmGcTime' # mediators
            ]
x = data.drop(drop_cols, axis=1).values
columns = data.drop(drop_cols, axis=1).columns

In [5]:
# l = []
# for item in data['spark.serializer']:
#     if item == 1:
#         l += [0]
#     else:
#         l += [1]
# data['spark.serializer'] = l
# data

In [7]:
np.random.seed(0)

In [8]:
x_train, x_test, y_train, y_test, treat_train, treat_test = train_test_split(x, y, treatment, test_size=0.2, random_state=42)

In [8]:
# regular CTL
ctl = CausalTree(cont=True)
ctl.fit(x_train, y_train, treat_train)
ctl.prune()
ctl_predict = ctl.predict(x_test)

# honest CTL
cth = CausalTree(cont=True, honest=True)
cth.fit(x_train, y_train, treat_train)
cth.prune()
cth_predict = cth.predict(x_test)

# val honest CTL
cthv = CausalTree(cont=True, val_honest=True)
cthv.fit(x_train, y_train, treat_train)
cthv.prune()
cthv_predict = cthv.predict(x_test)

# adaptive CT
ct_adaptive = CausalTree(weight=0.0, split_size=0.0, cont=True)
ct_adaptive.fit(x_train, y_train, treat_train)
ct_adaptive.prune()
ct_adaptive_predict = ct_adaptive.predict(x_test)

# honest CT
ct_honest = CausalTree(honest=True, weight=0.0, split_size=0.0, cont=True)
ct_honest.fit(x_train, y_train, treat_train)
ct_honest.prune()
ct_honest_predict = ct_honest.predict(x_test)


# if you want to plot a tree
# cthv.plot_tree(filename="output/all_configs_cont_trigger_tree")

# ct_adaptive.plot_tree(features=columns, filename="output/p_otherConfigs_cores/adaptive_ct_otherF", show_effect=True)
# ct_honest.plot_tree(features=columns, filename="output/p_otherConfigs_cores/honest_ct_otherF", show_effect=True)
# ctl.plot_tree(features=columns, filename="output/p_otherConfigs_cores/regular_ctl_otherF", show_effect=True)
# cth.plot_tree(features=columns, filename="output/p_otherConfigs_cores/honest_ctl_otherF", show_effect=True)
# cthv.plot_tree(features=columns, filename="output/p_otherConfigs_cores/valHonest_ctl_otherF", show_effect=True)


ct_adaptive.plot_tree(features=columns, filename="output/p_otherConfigs_memory/adaptive_ct_otherF", show_effect=True)
ct_honest.plot_tree(features=columns, filename="output/p_otherConfigs_memory/honest_ct_otherF", show_effect=True)
ctl.plot_tree(features=columns, filename="output/p_otherConfigs_memory/regular_ctl_otherF", show_effect=True)
cth.plot_tree(features=columns, filename="output/p_otherConfigs_memory/honest_ctl_otherF", show_effect=True)
cthv.plot_tree(features=columns, filename="output/p_otherConfigs_memory/valHonest_ctl_otherF", show_effect=True)

# ct_adaptive.plot_tree(features=columns, filename="output/otherConfigs_parallelism/parallelism_adaptive_ct_otherF", show_effect=True)
# ct_honest.plot_tree(features=columns, filename="output/otherConfigs_parallelism/parallelism_honest_ct_otherF", show_effect=True)
# ctl.plot_tree(features=columns, filename="output/otherConfigs_parallelism/parallelism_regular_ctl_otherF", show_effect=True)
# cth.plot_tree(features=columns, filename="output/otherConfigs_parallelism/parallelism_honest_ctl_otherF", show_effect=True)
# cthv.plot_tree(features=columns, filename="output/otherConfigs_parallelism/parallelism_valHonest_ctl_otherF", show_effect=True)

/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
# BINARY

# regular CTL
ctl = CausalTree(magnitude=False)
ctl.fit(x_train, y_train, treat_train)
ctl.prune()
ctl_predict = ctl.predict(x_test)

# honest CTL
cth = CausalTree(honest=True)
cth.fit(x_train, y_train, treat_train)
cth.prune()
cth_predict = cth.predict(x_test)

# val honest CTL
cthv = CausalTree(val_honest=True)
cthv.fit(x_train, y_train, treat_train)
cthv.prune()
cthv_predict = cthv.predict(x_test)

# adaptive CT
ct_adaptive = CausalTree(weight=0.0, split_size=0.0)
ct_adaptive.fit(x_train, y_train, treat_train)
ct_adaptive.prune()
ct_adaptive_predict = ct_adaptive.predict(x_test)

# honest CT
ct_honest = CausalTree(honest=True, weight=0.0, split_size=0.0)
ct_honest.fit(x_train, y_train, treat_train)
ct_honest.prune()
ct_honest_predict = ct_honest.predict(x_test)


# if you want to plot a tree
# cthv.plot_tree(filename="output/all_configs_cont_trigger_tree")

# ct_adaptive.plot_tree(features=columns, filename="output/otherConfigs_taskcpus/adaptive_ct_otherF", show_effect=True)
# ct_honest.plot_tree(features=columns, filename="output/otherConfigs_taskcpus/honest_ct_otherF", show_effect=True)
# ctl.plot_tree(features=columns, filename="output/otherConfigs_taskcpus/regular_ctl_otherF", show_effect=True)
# cth.plot_tree(features=columns, filename="output/otherConfigs_taskcpus/honest_ctl_otherF", show_effect=True)
# cthv.plot_tree(features=columns, filename="output/otherConfigs_taskcpus/valHonest_ctl_otherF", show_effect=True)


ct_adaptive.plot_tree(features=columns, filename="output/p_otherConfigs_serializer/adaptive_ct_otherF", show_effect=True)
ct_honest.plot_tree(features=columns, filename="output/p_otherConfigs_serializer/honest_ct_otherF", show_effect=True)
ctl.plot_tree(features=columns, filename="output/p_otherConfigs_serializer/regular_ctl_otherF", show_effect=True)
cth.plot_tree(features=columns, filename="output/p_otherConfigs_serializer/honest_ctl_otherF", show_effect=True)
cthv.plot_tree(features=columns, filename="output/p_otherConfigs_serializer/valHonest_ctl_otherF", show_effect=True)
